In [151]:
import urllib.request
import numpy as np
from lxml import html
from string import digits
import os

special_characters = ["\n", "\t", ",", "'", "-", ":", ";", "(", ")"]

if not os.path.isdir('books') :
    os.mkdir('books')


def get_author(htmltree):
    special_characters = ["\n", "\t", ",", "'", "-", ":", ";", "(", ")"]
    author = htmltree.xpath("//a[@itemprop='creator']/text()")
    author = "".join(author)
    
    for char in special_characters:
        author = author.replace(char, '')
        
    author = author.translate(remove_digits)
    
    return author

def get_title(htmltree):
    special_characters = ["\n", "\t", ",", "'", "-", ":", ";", "(", ")"]
        
    title = htmltree.xpath("//td[@itemprop='headline']/text()")
    title = "".join(title)
    
    for char in special_characters:
        title = title.replace(char, '')
    
    return title
    

remove_digits = str.maketrans('', '', digits)  

home_url = 'https://www.gutenberg.org'

# Load all urls of the top 100 downloaded books from gutenberg
url = 'https://www.gutenberg.org/browse/scores/top'

#Request the url and decode it to text
text = urllib.request.urlopen(url).read().decode('utf8')

#Generate a html tree to filter the proper hrefs
htmltree = html.fromstring(text)

#filter the hrefs from the page's first ordered list
hrefs = htmltree.xpath("/html/body/div/div[1]/ol[1]//a/@href")


for href in hrefs[60::]:
    
    bookinfo_url = home_url + href
    
    bookinfo = urllib.request.urlopen(bookinfo_url).read().decode('utf8')
    htmltree = html.fromstring(bookinfo)

    data = htmltree.xpath("//td/text()")
    
    filelink = [tc for tc in data if '.txt' in tc ]
    
    if not filelink:
        print('No .txt file for {}'.format(href))
        continue
    
    title = get_title(htmltree)
    author = get_author(htmltree)

    booktext = urllib.request.urlopen(filelink[0]).read().decode('utf8')

    filename = title.lower() + " " + author.lower()
    filename = filename.replace(" ", "-")[:-1] + ".txt"
    filename = os.path.join('./books', filename)
    
    txt_file = open(filename, "wt")
    n = txt_file.write(booktext)
    txt_file.close()

No .txt file for /ebooks/5740
